In [29]:
# import stanfordnlp
# stanfordnlp.download('en')
# nlp = stanfordnlp.Pipeline(processors = "tokenize,mwt,lemma,pos")
# doc = nlp("Event Management is the process that monitors all events that occur through the IT infrastructure to allow for normal operation and also to detect and escalate exception conditions.")
# doc.sentences[0].print_tokens()


In [30]:
# print(*[f'text: {word.text+" "}\tlemma: {word.lemma}\tupos: {word.upos}\txpos: {word.xpos}' for sent in doc.sentences for word in sent.words], sep='\n')

In [31]:
import Constants

with open("ITIL Books/ITIL 3/Service operation glossary/definitions.txt") as file:
    glossary_definitions = file.read()

# definitions = [i.lower().replace("\n", " ").split('::') for i in glossary_definitions.split('\n\n')]
definitions = [i.replace("\n", " ").split('::') for i in glossary_definitions.split('\n\n')]

concepts_from_definitions = [i[0].strip() for i in definitions]


with open("ITIL Books/ITIL 3/Service operation chapter 4/Service operation chapter 4 - all.txt") as file:
    text = file.read()

ontology = {}

for index in range(0, len(definitions)):
    ontology[definitions[index][0].strip()] = [f"definition::{definitions[index][1].strip()}"]

import nltk
# Tokenize into sentences
sentences = nltk.sent_tokenize(text)

# Split sentences into words
# sentences = [nltk.word_tokenize(i.lower().replace("\n", " ")) for i in sentences]
sentences = [nltk.word_tokenize(i.replace("\n", " ")) for i in sentences]

sentences_containing_concepts = {}
for sentence in sentences:
    # print("---")
    count = 0
    for concept in concepts_from_definitions:
        concept_list = concept.split()
        
        if concept_list[0] in sentence:
            add_to_sentences_containing_concepts = True
            
            if len(concept_list) > 1:
                current_word_index = sentence.index(concept_list[0])
                concept_word_length_count = 1
                for concept_word in concept_list[1::]:
                    current_word_index = current_word_index + 1
                    
                    if sentence[current_word_index] != concept_word:
                        add_to_sentences_containing_concepts = False
                        break
                
            if add_to_sentences_containing_concepts is True:
                sentence_string = " ".join(sentence)
                if sentence_string not in sentences_containing_concepts.keys():
                    sentences_containing_concepts[sentence_string] = []
                sentences_containing_concepts[sentence_string].append(concept)
            

# sentences_containing_concepts = [i for i in sentences if i in concepts_from_definitions]
 
# for sentence in sentences_containing_concepts:
#     print(sentence)

# print(sentences_containing_concepts)

# print(ontology)

for sentence, concepts in sentences_containing_concepts.items():
    for concept in concepts:
        ontology[concept].append(sentence)
        
# TODO: In definitions replace (book name) with own in book relation


In [58]:
import re
import random
import sys

def load_templates(file_path):
    with open(file_path) as template_file:
        templates = template_file.readlines()
    return templates

# question_template = "question_template(Question_number=1,Answer_number=4)::Question->What is the definition for {concept.name}?::Answer->{concept.definition}"
templates = load_templates("templates/question_templates.txt")
question_template = templates[random.randint(0, len(templates) - 1)]
# question_template = templates[1]

# print(templates)
# print(question_template)

# nltk.tokenize.mwe 
        
def get_definition(concept_name):
    return [i.split("::")[1] for i in ontology[concept_name] if i.split("::")[0] == "definition"][0]

class Concept:
    def __init__(self, name, ontology):
        self.name = name
        self.ontology = ontology
        self.attributes = ontology[self.name]
        self.definition = get_definition(self.name)
        
        # if hasattr(self, 'book') and not self.book:
        self.book = re.findall("^\w*\(.*?\)", self.definition)
    
        for book in self.book:
            self.definition = self.definition.replace(book, "")
        
        self.book = [i.replace("(", "").replace(")", "") for i in self.book]
        self.attributes[0] = f"definition::{self.definition}"
        # TODO: Fix book

class Question:
    def __init__(self, question_string, answer_list, question_params):
        self.question_string = question_string
        self.answer_list = answer_list
        self.parameters = question_params
        self.correct_answer = answer_list[0]
        
    def randomise_answers(self):
        # self.correct_answer = self.answer_list[0]
        random.shuffle(self.answer_list)
        return self.answer_list.index(self.correct_answer)
    
    def print_question(self):
        print(self.question_string)
        
    def print_answers(self):
        for answer_index, answer in enumerate(self.answer_list):
            print(f"- {answer_index + 1}: " + answer.strip("\n"))
    
    # def print_question_and_answer(self):        
    #     print(self.question_string)
    #     self.randomise_answers()
    #     for answer in self.answer_list:
    #         print(f"- " + answer.strip("\n"))
    #     print()
        
    def format_output_print(self):
        self.print_question()
        self.randomise_answers()
        self.print_answers()
        print()
        
    def format_output_interactive(self):
        self.print_question()
        correct_answer_index = self.randomise_answers()
        self.print_answers()
        
        while(True):
            try:
                answer_input = int(input("What is the answer (answer ID number) [0 for exit]?"))
                print(answer_input)
                if answer_input == 0:
                    break
                elif 0 < answer_input <= len(self.answer_list):
                    if answer_input == (correct_answer_index + 1):
                        print(f"Correct")
                        break
                    else:
                        print(f"Incorrect\nThe correct answer is {self.answer_list[correct_answer_index]}")
                else:
                    print(f"Answer ID [{answer_input}] is not recognised")
            except (ValueError):
                print(f"The inputted answer is not a question number, please enter the answer ID (e.g. 1)")
                
            print()
               
    def format_output_return(self):
        output = []
        output.append(self.question_string)
        self.randomise_answers()
        for answer_index, answer in enumerate(self.answer_list):
            output.append(f"- {answer_index + 1}: " + answer.strip("\n"))
        return output
        
    def format_output(self, output_type="print"):
        if output_type.lower() == "print":
            self.format_output_print()
        elif output_type.lower() == "interactive":
            self.format_output_interactive()
        elif output_type.lower() == "return":
            return self.format_output_return()
        else:
            self.format_output_print()
        
        
def get_incorrect_answers(correct_concept, question_parameters):
    # TODO: Move literal strings into a central Constants file so they can be easily changed
    number_of_answers = question_parameters["Answer_number"] if "Answer_number" in question_parameters.keys() else 4
    answer_type = question_parameters["Answer_type"] if "Answer_type" in question_parameters.keys() else None
    
    incorrect_answers = []
    temp_books = [i for i in Constants.ITIL_3_BOOKS if i not in correct_concept.book]
    # print(temp_books)
    
    while(len(incorrect_answers) < int(number_of_answers) - 1):
        # print(f"ontology.keys: {ontology.keys()}")
        # random_int = random.randint(0, len(ontology))
        # print(f"random_int: {random_int}")
        # print(f"len(ontology): {len(ontology)}")
        # 
        # incorrect_concept_name = list(ontology.keys())[random_int]
        try:
            incorrect_concept_name = list(ontology.keys())[random.randint(0, len(ontology.keys())) - 1]
            
            
            if incorrect_concept_name != correct_concept.name and answer_type == "definition":
                incorrect_answers.append(get_definition(incorrect_concept_name))
            elif incorrect_concept_name != correct_concept.name and answer_type == "concept.name":
                incorrect_answers.append(incorrect_concept_name)
            elif incorrect_concept_name != correct_concept.name and answer_type == "books":
                if len(temp_books) > 0:
                    book = temp_books[random.randint(0, len(temp_books) - 1)]
                    temp_books.remove(book)
                    incorrect_answers.append(book)
                else:
                    break
                # print("-" * 100)
                # print(correct_concept.book)
                # print(Constants.ITIL_3_BOOKS)
                # print(temp_books)
                # temp_books = [i for i in correct_concept.book]
                # incorrect_answers.append(ITIL_3_BOOKS[])
        except IndexError:
            continue
    
    # TODO: Make answer picker more generalised
    
    return incorrect_answers
    
def get_random_attribute_without_concept_name(concept):
    random_attribute = concept.attributes[random.randint(0, len(concept.attributes) - 1)].replace(concept.name, "_" * 10)
    # print(random_attribute)
    
    if "definition::" in random_attribute:
        random_attribute = f"The definition of {'_' * 10} is: {random_attribute.replace('definition::', '')}"
        
    # return concept.attributes[].replace("{!concept.name}", "_" * 10)
    return random_attribute
    
def replace_template_variables(template_format_string, concept):
    replaced_string = template_format_string
    
    # print(replaced_string)
    
    # get_random_attribute_without_concept_name(concept)
    
    if "{!concept.name}" in replaced_string:
        replaced_string_replacer = get_random_attribute_without_concept_name(concept)
        replaced_string = replaced_string.replace("{!concept.name}", replaced_string_replacer)
    
    # replaced_string = replaced_string.replace("{!concept.name}", "_" * 10)
    replaced_string = replaced_string.replace("{newline}", "\n")
    replaced_string = replaced_string.replace("{concept.name}", concept.name)
    replaced_string = replaced_string.replace("{concept.definition}", concept.definition)
    if len(concept.book) > 0:
        replaced_string = replaced_string.replace("{concept.book}", concept.book[random.randint(0, len(concept.book) - 1)])
    else:
        replaced_string = replaced_string.replace("{concept.book}", "Not in a specific book")
    
    # replaced_string = re.sub("{.*?\!concept\.name.*?}", "__________", replaced_string)
    # replaced_string = re.sub("{.*?newline}.*?", "\n", replaced_string)
    # replaced_string = re.sub("{.*?concept\.name.*?}", concept.name, replaced_string)
    # replaced_string = re.sub("{.*?concept\.definition.*?}", concept.definition, replaced_string)\
    # replaced_string = re.sub("{.*?concept\.relation\.attribute*?}", concept.attribute, replaced_string)
    # replaced_string = re.sub("{.*?concept\.book.*?}", concept.book, replaced_string)
    
    # print(replaced_string)
    
    return replaced_string

def parse_template_question(question_template_to_parse, concept_name="", debug=False):
    template_list = question_template_to_parse.split("::")
    question_parameters = template_list[0].split("(")[1].replace(")", "").split(",")    
    question_parameters = {item.split("=")[0]: item.split("=")[1] for item in question_parameters}
    question = template_list[1].split("->")[1]
    answer = template_list[2].split("->")[1]
            
    if concept_name == "":
        concept_name = list(ontology.keys())[random.randint(0, len(ontology)) - 1]  
    else:
        if concept_name not in ontology.keys():
            print(f"The concept [{concept_name}] was not a recognised concept in the ontology")
            sys.exit(0)
                
    main_concept = Concept(concept_name, ontology)
    
    if debug is True:
            print(f"""
{"-" * 100}
Concept:
    Name = {main_concept.name}
    Definition = {main_concept.definition}
    Books = {main_concept.book}
    Attributes = {main_concept.attributes}
{"-" * 100}
            """)
        
    question = replace_template_variables(question, main_concept)
    answer = replace_template_variables(answer, main_concept)
    
    answer_list = [answer]
                
    answer_list = answer_list + get_incorrect_answers(main_concept, question_parameters)
            
    return Question(question, answer_list, question_parameters)

def parse_template(template_to_parse, concept_name="", debug=False):
    if "question_template" in template_to_parse.split("::")[0]:
        return parse_template_question(template_to_parse, concept_name=concept_name, debug=debug)
    else:
        print(f"Error parsing template {template_to_parse}")
        return None
    

In [54]:
parsed_template = parse_template(question_template, concept_name="Access Management", debug=True)
 
# parsed_template = parse_template(question_template)

parsed_template.format_output(output_type="print")

# for sentence in sentences:
#     print(nltk.word_tokenize(sentence))
    
# print(nltk.tokenize.TextTilingTokenizer().tokenize(glossary_definitions))
# print(nltk.word_tokenize(glossary_definitions))



----------------------------------------------------------------------------------------------------
Concept:
    Name = Access Management
    Definition =  The Process responsible for allowing Users to make use of IT Services, data, or other Assets. Access Management helps to protect the Confidentiality, Integrity and Availability of Assets by ensuring that only authorized Users are able to access or modify the Assets. Access Management is sometimes referred to as Rights Management or Identity Management.
    Books = []
    Attributes = ['definition:: The Process responsible for allowing Users to make use of IT Services, data, or other Assets. Access Management helps to protect the Confidentiality, Integrity and Availability of Assets by ensuring that only authorized Users are able to access or modify the Assets. Access Management is sometimes referred to as Rights Management or Identity Management.', 'ï¿½ Access Management : this is the process of granting authorized users the right

In [34]:
# for concept in ontology:
#     print(concept)
#     for relationship in ontology[concept]:
#         print(f"\t-{relationship}")



In [35]:
# # Code from https://spacy.io/
# import spacy
# 
# nlp = spacy.load("en_core_web_sm")
# 
# text = "Event Management is the process that monitors all events that occur through the IT infrastructure to allow for normal operation and also to detect and escalate exception conditions."
# 
# doc = nlp(text)
# 
# # # Analyze syntax
# print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
# print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])
# 
# print("-" * 100)
# 
# # Find named entities, phrases and concepts
# for entity in doc.ents:
#     print(entity.text, entity.label_)
# 
# print("-" * 100)
# for token in doc:
#     print(token.text, token.pos_, token.dep_)
# 
# print("-" * 100)
# for token in doc:
#     print(token.text, token.dep_)

# from spacy import displacy
# displacy.render(doc, style="dep", jupyter=True, options={'distance': 140})


In [93]:
# for concept in ontology:
#     print(concept)attribute.replace

# print(ontology["service"][1])


def generate_parsed_templates(number_of_questions=1, concept_name = "", debug=False, random_templates=True, template_to_use=""):
    parsed_templates = []
    
    for i in range(0, number_of_questions):
        if random_templates is True and template_to_use is "":
            question_template = templates[0] if i is 0 else templates[random.randint(1, len(templates) - 1)]
        elif random_templates is False and template_to_use is not "":
            question_template = templates[int(template_to_use)]
        else:
            if i == 0:
                question_template = templates[0]
            elif i == 1:
                question_template = templates[2]
            else:
                question_template = templates[1]
        # question_template = templates[2]
        parsed_templates.append(parse_template(question_template, concept_name=concept_name, debug=debug if i is 0 else False))
    return parsed_templates
        
def output_to_file(filename, questions, answers, title=""):
    print(f"Printing answers and questions to file {filename}")
    
    with open(f"{filename}-questions.txt", "w+") as file:
        file.write("-" * 100 + "\n")
        file.write(f"{title}\n")
        file.write("-" * 100 + "\n")
        
        for question_number, question in enumerate(questions):
            file.write(f"Question {question_number}:\n")
            for line in question:
                file.write(f"{line}\n")
            file.write("-" * 100 + "\n")

    with open(f"{filename}-answers.txt", "w+") as file:
        file.write("-" * 100 + "\n")
        file.write(f"{title}\n")
        file.write("-" * 100 + "\n")
        
        for question_number, answer in enumerate(answers):
            file.write(f"Question {question_number}:\n{answer}")
            file.write("-" * 100 + "\n")
            
    print(f"Printed answers and questions to file {filename}")
        
                
### Generate 100 random questions using random templates ###
# parsed_templates = generate_parsed_templates(number_of_questions=100, concept_name="", random_templates=True)
# for parsed_template in parsed_templates:
#     parsed_template.format_output()
#     print(f"Correct answer: {parsed_template.correct_answer}")
#     print("-" * 100)
    
### Generate 25 random questions using template 0 ###
title = "25 Template 0 questions"
questions = []
answers = []
parsed_templates = generate_parsed_templates(number_of_questions=25, concept_name="", random_templates=False, template_to_use=0)
for parsed_template in parsed_templates:
    questions.append(parsed_template.format_output(output_type="return"))
    answers.append(parsed_template.correct_answer)
    
output_to_file(f"Output/{title}", questions, answers, title=title)
    
# ### Generate 25 random questions using template 1 ###
title = "25 Template 1 questions"
questions = []
answers = []
parsed_templates = generate_parsed_templates(number_of_questions=25, concept_name="", random_templates=False, template_to_use=1)
for parsed_template in parsed_templates:
    questions.append(parsed_template.format_output(output_type="return"))
    answers.append(parsed_template.correct_answer)
    
output_to_file(f"Output/{title}", questions, answers, title=title)

### Generate 25 random questions using template 2 ###
title = "25 Template 2 questions"
questions = []
answers = []
parsed_templates = generate_parsed_templates(number_of_questions=25, concept_name="", random_templates=False, template_to_use=2)
for parsed_template in parsed_templates:
    questions.append(parsed_template.format_output(output_type="return"))
    answers.append(parsed_template.correct_answer)
    
output_to_file(f"Output/{title}", questions, answers, title=title)
    
# parsed_template = parse_template(question_template, concept_name="Event Management", debug=True)
# parsed_template.format_output()

# for concept in ontology:
#     # concept = ontology["service"]
#     for index, sentence in enumerate(ontology[concept]):
#         # print(sentence)
#         # print(index)
#         # print(concept)
#         if "definition::" in sentence:
#             continue
#             
#         # print(sentence.replace("service", "{concept}"))
#         ontology[concept][index] = sentence.replace(concept, "{!concept.name}")
#                 
#         # print(sentence.concept.keys()[0])
        
# concept_name = "service"
# print(get_random_attribute_without_concept_name(Concept(concept_name, ontology)))


Printing answers and questions to file Output/25 Template 0 questions
Printed answers and questions to file Output/25 Template 0 questions
Printing answers and questions to file Output/25 Template 1 questions
Printed answers and questions to file Output/25 Template 1 questions
Printing answers and questions to file Output/25 Template 2 questions
Printed answers and questions to file Output/25 Template 2 questions


# Unittests


In [37]:
import unittest

class TestUnitTests(unittest.TestCase):

    def test_generate_parsed_templates(self):
        parsed_templates = generate_parsed_templates(1, debug=False)[0]
        self.assertIsInstance(parsed_templates, Question)
        
    def test_parse_template_question(self):
        try:
            parse_template_question(templates[0])
        except IndexError:
            self.fail("myFunc() raised ExceptionType unexpectedly!")        
        
    ### Testing classes ###
    # Question
    def test_Question(self):
        question = Question(
            "Question example unit test 1",
            ["Answer 1", "Answer 2", "Answer 3", "Answer 4"],
            {'Question_number': '1', 'Answer_number': '4', 'Question_type': 'attribute.replace', 'Answer_type': 'concept.name'})
        self.assertEqual(question.question_string, "Question example unit test 1")
        self.assertEqual(question.answer_list, ["Answer 1", "Answer 2", "Answer 3", "Answer 4"])
        self.assertEqual(question.parameters, {'Question_number': '1', 'Answer_number': '4', 'Question_type': 'attribute.replace', 'Answer_type': 'concept.name'})
        self.assertEqual(question.correct_answer, "Answer 1")
        
    # Concept
    def test_Concept(self):
        ontology = {
            'Acceptance': 
                ['definition::Formal agreement that an IT Service, Process, Plan, or other Deliverable is complete, accurate, Reliable and meets its specified Requirements. Acceptance is usually preceded by Evaluation or Testing and is often required before proceeding to the next stage of a Project or Process.',
                 'Other Problem Records , and corresponding Known Error Records , may be triggered in testing , particularly the latter stages of testing such as User Acceptance Testing/Trials ( UAT ) , if a decision is made to go ahead with a release even though some faults are known .'],
            'Access Management':
                ['definition::(Service Operation) The Process responsible for allowing Users to make use of IT Services, data, or other Assets.', 'ï¿½ Access Management : this is the process of granting authorized users the right to use a service , while restricting access to non-authorized users .']
        }
        
        concept_1 = Concept(
            "Acceptance",
            ontology
        )
        
        self.assertEqual(concept_1.name, "Acceptance")
        self.assertEqual(concept_1.ontology, ontology)
        self.assertFalse(concept_1.book)
        
        concept_2 = Concept(
            "Access Management",
            ontology
        )
        
        self.assertEqual(concept_2.name, "Access Management")
        self.assertEqual(concept_2.ontology, ontology)
        self.assertIsInstance(concept_2.book, list)
        # self.assertEqual(concept_2.book, ["Service Operation"])

unittest.main(argv=[''], verbosity=2, exit=False)

test_Concept (__main__.TestUnitTests) ... ok
test_Question (__main__.TestUnitTests) ... ok
test_generate_parsed_templates (__main__.TestUnitTests) ... ok
test_parse_template_question (__main__.TestUnitTests) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK
